In [12]:
from huggingface_hub import login
from datasets import load_dataset
import numpy as np
import librosa
import math
import pickle

/mnt/adamn/anaconda3/envs/inz-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("mozilla-foundation/common_voice_13_0", "en", cache_dir='/mnt/adamn/abc/inzynierka/data/CommonVoice')

Found cached dataset common_voice_13_0 (/mnt/adamn/abc/inzynierka/data/CommonVoice/mozilla-foundation___common_voice_13_0/en/13.0.0/22809012aac1fc9803eaffc44122e4149043748e93933935d5ea19898587e4d7)
100%|██████████| 5/5 [00:00<00:00, 39.26it/s]


In [3]:
train = ds['train']

In [11]:
train[0]

{'client_id': '09cf128e35eee9b49f71a4c58d893f1a5131b5047474704808317562d522272df611f4d104e28a50112fa4002cf4eb2f4520fab52c4b9401313960c12e32a459',
 'path': '/mnt/adamn/abc/inzynierka/data/CommonVoice/downloads/extracted/cf9bb4c3bced3afc7bf434f599d4f9dde2a5ed6da50e015d21014df777dd70b9/common_voice_en_21635524.mp3',
 'audio': {'path': '/mnt/adamn/abc/inzynierka/data/CommonVoice/downloads/extracted/cf9bb4c3bced3afc7bf434f599d4f9dde2a5ed6da50e015d21014df777dd70b9/common_voice_en_21635524.mp3',
  'array': array([ 0.00000000e+00, -1.64753278e-14, -5.40568024e-14, ...,
          7.96796940e-08, -1.15487887e-06, -8.86387454e-07]),
  'sampling_rate': 48000},
 'sentence': 'This device has a cathode inside an anode wire cage.',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'en',
 'segment': '',
 'variant': ''}

In [10]:
lens = []
for i in range(100):
    a = audio_to_mfcc(train[i]['audio']['array'], train[i]['audio']['sampling_rate'])
    np.save('./plik.npy', a)
    l = os.stat('plik.npy').st_size/1024/1024/1024
    lens.append(l)

In [68]:
1013968 / 100 * np.mean(lens)

5.9417702394723895

In [6]:
def audio_to_mfcc(audio, sr):
        S = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1024, hop_length=256)
        mfcc = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=23)
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        frame_energy = librosa.feature.rms(y=audio, frame_length=1024, hop_length=256)
        feature_vector = np.vstack((mfcc, mfcc_delta, mfcc_delta2, frame_energy))
        v_a, v_b = feature_vector.shape
        return np.reshape(feature_vector, (1, v_a, v_b))

In [11]:
len(train)

1013968

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1013968
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 16372
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 16372
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 278333
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        n

In [13]:
for i in ds:
    print(i)

train
validation
test
other
invalidated


In [15]:
import multiprocessing as mp

In [33]:
pool = mp.Pool(3)

In [34]:
def multi(a):
    return a**2

In [35]:
results = []

In [36]:
data = [1,2,3,1,2,22,74,2,4,11]*1000

In [37]:
for d in data:
    result = pool.apply_async(multi, args=(d,))
    results.append(result)

In [41]:
len([i.get() for i in results])

10000

In [39]:
pool.close()
pool.join()

In [43]:
len(results)

10000

In [1]:
import numpy as np

In [44]:
a = np.array([[1,2,5,2]*10000]*70)
np.save('plik.npy', a)

In [70]:
np.load('plik.npy').shape

(1, 70, 1027)

In [49]:
os.stat('plik.npy').st_size/1024/1024 * 1013968 / 1024

21153.141733169556

In [1]:
import os

In [3]:
root = '/mnt/adamn/abc/inzynierka/data/transformed_cv/'
for f_name in os.listdir(root):
    print(f_name)
    print(len(os.listdir(os.path.join(root, f_name))))

60_female
32682
30_male
109336
40_male
85147
20_female
77625
10_male
38385
20_male
196567
60_male
26789
10_female
22344
40_female
22275
50_male
41699
30_female
21463
50_female
17627


In [2]:
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [3]:
def loader(path):
    return np.load(path)

extensions = ['.npy']

In [4]:
root = '/mnt/adamn/abc/inzynierka/data/transformed_cv/data'

ds = DatasetFolder(root, loader, extensions)
train_size = int(0.8 * len(ds))  # 70% danych jako zbiór treningowy
val_size = int(0.1 * len(ds))  # 15% danych jako zbiór walidacyjny
test_size = len(ds) - train_size - val_size  # Pozostałe 15% danych jako zbiór testowy
train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])

In [15]:
a = DataLoader(
    dataset=test_dataset,
    batch_size=10,
    shuffle=True,
    collate_fn=collate_fn
    )

In [19]:
for i in a:
    print(type(i))
    break

<class 'tuple'>


In [10]:
a.dataset.dataset.classes

['10_female',
 '10_male',
 '20_female',
 '20_male',
 '30_female',
 '30_male',
 '40_female',
 '40_male',
 '50_female',
 '50_male',
 '60_female',
 '60_male']

In [17]:
import torch

In [14]:
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: x[0].shape[2], reverse=True)
    max_length = batch[0][0].shape[2]
    padded_batch = []
    outputs = []
    for sample in batch:
        length = sample[0].shape[2]
        padding_length = max_length - length
        padding = torch.zeros((1, 70, padding_length))
        padded_sample = torch.cat([torch.Tensor(sample[0]), padding], dim=2)
        padded_batch.append(padded_sample.unsqueeze(1))
        outputs.append(torch.Tensor([sample[1]]))
    return (torch.cat(padded_batch, dim=0), torch.cat(outputs, dim=0))

In [105]:
a = DataLoader(
    dataset=test_dataset,
    batch_size=10,
    shuffle=True,
    collate_fn=collate_fn
    )

In [107]:
for i in a:
    print(i[1])
    break

tensor([ 7., 10.,  5.,  3.,  5.,  3.,  7.,  3.,  2.,  7.])


In [110]:
list(range(12))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [111]:
indices = list(range(len(ds)))

In [113]:
class_indices = {i: list() for i in range(12)}

In [118]:
for i in indices:
    _, label = ds[i]  # Zakładając, że otrzymujemy etykiety dla próbek
    class_indices[label].append(i)
    if i%10000==0: print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000


In [121]:
train_indices = []
val_indices = []
test_indices = []

train_size_per_class = {0: 15000, 1: 25000, 2: 15000, 3: 25000, 4: 15000, 5: 25000,
                        6: 15000, 7: 25000, 8: 15000, 9: 25000, 10: 15000, 11: 25000}
val_size_per_class = 1000
test_size_per_class = 1000

random.seed(42)
for class_label, indices in class_indices.items():
    random.shuffle(indices)
    train_indices.extend(indices[:train_size_per_class[class_label]])
    val_indices.extend(indices[train_size_per_class[class_label]:train_size_per_class[class_label] + val_size_per_class])
    test_indices.extend(indices[train_size_per_class[class_label] + val_size_per_class:train_size_per_class[class_label] + val_size_per_class + test_size_per_class])

train_dataset = Subset(ds, train_indices)
val_dataset = Subset(ds, val_indices)
test_dataset = Subset(ds, test_indices)

In [ ]:
with open('/mnt/adamn/abc/inzynierka/data/transformed_cv/subsets/train_indices.pkl', 'wb') as file:
    pickle.dump(train_indices, file)

with open('/mnt/adamn/abc/inzynierka/data/transformed_cv/subsets/val_indices.pkl', 'wb') as file:
    pickle.dump(val_indices, file)

with open('/mnt/adamn/abc/inzynierka/data/transformed_cv/subsets/test_indices.pkl', 'wb') as file:
    pickle.dump(test_indices, file)

In [127]:
len(test_indices)

11789

In [124]:
train_dataset.indices

[18290,
 3978,
 9545,
 2978,
 4048,
 16107,
 13084,
 21820,
 5238,
 18908,
 856,
 14866,
 20976,
 5444,
 17508,
 1370,
 17044,
 9054,
 18126,
 15634,
 15949,
 21912,
 19528,
 16652,
 6477,
 7788,
 15168,
 2917,
 6754,
 6927,
 5886,
 14757,
 12140,
 7034,
 210,
 5918,
 7996,
 9009,
 1880,
 7705,
 17635,
 2320,
 9001,
 10665,
 21714,
 2919,
 5183,
 3747,
 14311,
 2775,
 14146,
 2268,
 2791,
 2370,
 8648,
 8658,
 1887,
 15514,
 12962,
 7274,
 3499,
 2490,
 16466,
 12551,
 10899,
 9187,
 8287,
 5109,
 11401,
 16361,
 7081,
 14533,
 21720,
 11242,
 273,
 14001,
 20293,
 10868,
 4155,
 4337,
 10160,
 14402,
 5071,
 17782,
 20046,
 19057,
 6943,
 6829,
 7977,
 15103,
 9830,
 18558,
 21027,
 3274,
 6337,
 18786,
 1868,
 7302,
 10013,
 11373,
 3268,
 6396,
 18078,
 12115,
 8639,
 4037,
 2631,
 15467,
 6549,
 11191,
 9612,
 1673,
 13913,
 4037,
 14611,
 14239,
 13006,
 19496,
 12810,
 5311,
 13293,
 8442,
 16746,
 11091,
 3623,
 12117,
 20693,
 14214,
 5441,
 5057,
 9803,
 17117,
 20014,
 1912,


In [119]:
import random
from torch.utils.data import DataLoader, Dataset, Subset, random_split